# Projeto SuperComputação 2024.1

## Otimização de Rotas de Veículos (_Vehicle Routing Problem_ - VRP)

**Objetivo:**

Desenvolver uma aplicação em C++ que otimize as rotas de uma frota de veículos destinados à entrega de produtos a diversos clientes, buscando minimizar o custo total das rotas.


**Descrição:**

Você está encarregado de criar uma solução computacional para o problema de Otimização de Rotas de Veículos (VRP) para uma empresa de logística. A empresa possui uma frota de veículos que são usados para entregar produtos a uma série de clientes. Cada veículo tem uma capacidade de carga máxima, e cada cliente tem uma demanda específica que deve ser atendida. O objetivo é determinar as rotas ótimas para os veículos, de forma que todos os clientes sejam atendidos, respeitando as restrições de capacidade dos veículos, e minimizando o custo total das entregas.


**Restrições e Requisitos:**
1. **Capacidade dos Veículos:** Cada veículo tem uma capacidade máxima de carga que não pode ser excedida.
2. **Demanda dos Clientes:** Cada cliente tem uma demanda específica que deve ser completamente atendida por um único veículo.
3. **Custo da Rota:** Cada rota possui um custo, que simboliza o cômputo agregado da distância percorrida, do tempo de entrega, e do consumo de combustível. O objetivo é minimizar o custo total.
4. **Número máximo de visitas por rota:** Cada rota pode visitar um número máximo de cidades.
5. **Ponto de Partida e Chegada:** Todos os veículos começam e terminam suas rotas no depósito da empresa.


**Desafios Computacionais:**

- O VRP é um problema NP-difícil, o que significa que sua complexidade computacional cresce exponencialmente com o aumento do número de clientes e veículos.
- A busca exaustiva por todas as possíveis combinações de rotas torna-se impraticável para instâncias maiores do problema.


**Composição da Solução:**
1. Implementação de um algoritmo de _busca global_ para o problema (força bruta);
2. Implementação de alguma solução aproximada (_heurística_ ou _busca local_);
3. Implementação de 2 soluções paralelizadas, sendo:

  a. _Paralelização com Threads OpenMP_: Utilizar threads e a biblioteca OpenMP para paralelizar o algoritmo desenvolvido, com o objetivo de reduzir o tempo de execução. Analisar e implementar a paralelização de componentes do algoritmo que possam se beneficiar da execução concorrente, como a geração de soluções iniciais ou a busca local.

  b. _Paralelização e Distribuição do Processamento com MPI_: Implementar uma versão do algoritmo que utilize a interface de passagem de mensagens MPI para distribuir o processo de busca de soluções em múltiplos processos, possivelmente executando em diferentes nós de um cluster. A estratégia deve permitir a exploração paralela do espaço de soluções e a troca eficiente de informações entre os processos.



**Entregáveis:**

1. **Código Fonte:** Código fonte em C++ comentado e organizado.
2. **Relatório:** Um relatório descrevendo a abordagem utilizada, incluindo descrição das heurísticas e métodos de busca local, bem como as estratégias de paralelização adotadas.
3. **Análise de Desempenho:** Uma análise de desempenho da solução proposta, incluindo tempos de execução e qualidade das soluções encontradas, com base em instâncias de teste de diferentes tamanhos.
4. **Instruções de Uso:** Breve documentação sobre como compilar e executar a aplicação, incluindo exemplos de uso.


**DICA**:
Não deixe de estudar os exemplos deste link https://vrpy.readthedocs.io/en/master/examples.html, para entender como gerar a avaliar soluções para o problema. Compare sua solução com a solução desta biblioteca!

### GERAÇÃO DE INSTÂNCIAS DO PROBLEMA

Execute o código abaixo algumas vezes gerando instâncias de tamanhos e complexidades distintas. Você deve experimentar alterações de:
- **num_nos**: número de cidades a serem visitados;
- **probabilidade**: a probabilidade de ser criada uma rota direta entre 2 cidades;


In [29]:
import random
# from networkx import set_node_attributes

def gerar_dicionario_demandas(N):
    """
    Gera um dicionário onde a chave é um int de 1 até N e o valor é um inteiro aleatório de 1 até 10.

    :param N: Número máximo para as chaves do dicionário.
    :return: Dicionário com chaves de 1 até N e valores inteiros aleatórios de 1 até 10.
    """
    return {i: random.randint(1, 10) for i in range(1, N)}


def gerar_entradas_grafo(num_nos, max_peso=100, probabilidade=0.25):
    """
    Gera um grafo para o problema de otimização de rotas de veículos.

    :param num_nos: Número de nós no grafo, incluindo o depósito.
    :param max_peso: Peso máximo para as arestas do grafo.
    :param probabilidade: Probabilidade de criar uma rota entre duas cidades.
    :return: Um dicionário representando o grafo onde as chaves são tuplas representando as arestas (nó1, nó2)
             e os valores são os pesos dessas arestas.
    """
    grafo = {}
    # Gerar pesos para arestas entre o depósito e outros nós
    for i in range(1, num_nos):
        grafo[(0, i)] = random.randint(1, max_peso)
        grafo[(i, 0)] = grafo[(0, i)]  # Assume que a distância de volta ao depósito é a mesma

    # Gerar pesos para arestas entre todos os outros pares de nós
    for i in range(1, num_nos+1):
        for j in range(i+1, num_nos):
            if random.random() > (1 - probabilidade):  # Verifica a probabilidade
                peso = random.randint(1, max_peso)
                grafo[(i, j)] = peso

    return grafo

############################################
#             Exemplo de uso
############################################
num_nos = 10                                   # Número total de nós incluindo o depósito
demandas = gerar_dicionario_demandas(num_nos)  # Gera as demandas para cada nó
grafo = gerar_entradas_grafo(num_nos)          # Gera o grafo que representa os locais e custos entre eles

# Salva o grafo em um arquivo TXT
with open('grafo.txt', 'w') as arquivo:
  arquivo.write(str(num_nos) + "\n")    # Número de nós, incluindo depósito
  for local, demanda in demandas.items():
    linha = f"{local} {demanda}\n"      # Par LOCAL DEMANDA
    arquivo.write(linha)

  arquivo.write(str(len(grafo)) + "\n") # Número de arestas
  for aresta, peso in grafo.items():
    linha = f"{aresta[0]} {aresta[1]} {peso}\n" # Trio: ORIGEM DESTINO CUSTO
    arquivo.write(linha)


### GERAÇÃO DE SOLUÇÕES

Instale a biblioteca [VRPy](https://vrpy.readthedocs.io/en/latest/getting_started.html) para resolver problemas de otimização de rotas.

Para cada instância do problema gerada, execute este código para gerar a solução esperada.

Não deixe de estudar a documentação da biblioteca! Especialmente este link: https://vrpy.readthedocs.io/en/latest/examples.html#a-simple-example

Aqui você pode gerar soluções diferentes alterando:
- **load_capacity**: a quantidade e capacidade de veículos;
- **num_stops**: o número máximo de paradas numa rota;

In [15]:
!pip install vrpy

Defaulting to user installation because normal site-packages is not writeable


In [30]:
from networkx import DiGraph, set_node_attributes
from vrpy import VehicleRoutingProblem


def ler_arquivo_grafo(caminho_arquivo):
    with open(caminho_arquivo, 'r') as arquivo:
        # Lê o número de nós
        N = int(arquivo.readline().strip())-1

        # Lê as demandas dos nós
        demandas = {}
        for _ in range(N):
            linha = arquivo.readline().strip().split()
            id_no, demanda = int(linha[0]), int(linha[1])
            demandas[id_no] = demanda

        # Lê o número de arestas
        K = int(arquivo.readline().strip())

        # Lê as arestas
        arestas = []
        for _ in range(K):
            linha = arquivo.readline().strip().split()
            origem, destino, peso = int(linha[0]), int(linha[1]), int(linha[2])
            arestas.append((origem, destino, peso))

    return demandas, arestas

############################################
#             Usando a funcao
############################################
caminho_arquivo = 'grafo.txt'
demandas, arestas = ler_arquivo_grafo(caminho_arquivo)

G = DiGraph()
for inicio, fim, custo in arestas:
    if inicio==0: inicio="Source"
    if fim==0: fim="Sink"
    G.add_edge(inicio, fim, cost=custo)

set_node_attributes(G, values=demandas, name="demand")

prob = VehicleRoutingProblem(G, load_capacity=15)  # Pode alterar a capacidade
prob.num_stops = 5                                 # Pode alterar o número máximo de paradas
prob.solve()

INFO:vrpy.vrp:new upper bound : max num stops = 5
INFO:vrpy.vrp:Clarke & Wright solution found with value 598 and 6 vehicles
INFO:vrpy.vrp:Greedy solution found with value 639 and 7 vehicles
INFO:vrpy.vrp:iteration 0, 598.0
INFO:vrpy.master_solve_pulp:total cost = 598.0


No negative cost cycle has been found and elementary set to true.
Consider setting elementary to false.
No negative cost cycle has been found and elementary set to true.
Consider setting elementary to false.
No negative cost cycle has been found and elementary set to true.
Consider setting elementary to false.
No negative cost cycle has been found and elementary set to true.
Consider setting elementary to false.
No negative cost cycle has been found and elementary set to true.
Consider setting elementary to false.
No negative cost cycle has been found and elementary set to true.
Consider setting elementary to false.
No negative cost cycle has been found and elementary set to true.
Consider setting elementary to false.
No negative cost cycle has been found and elementary set to true.
Consider setting elementary to false.
No negative cost cycle has been found and elementary set to true.
Consider setting elementary to false.
No negative cost cycle has been found and elementary set to true

In [31]:
prob.best_routes

{1: ['Source', 6, 'Sink'],
 2: ['Source', 4, 8, 'Sink'],
 3: ['Source', 1, 'Sink'],
 4: ['Source', 2, 9, 'Sink'],
 5: ['Source', 5, 7, 'Sink'],
 6: ['Source', 3, 'Sink']}

In [32]:
prob.best_value

598

In [33]:
prob.best_routes_cost

{1: 144, 2: 43, 3: 134, 4: 131, 5: 74, 6: 72}

In [25]:
prob.best_routes_load

{1: 9, 2: 6, 3: 11, 4: 1, 5: 7, 6: 2}

 ### 1. Algoritmo de busca global

Dado o pseudo-código abaixo, implemente-o em C++ e compare (com um gráfico) seu tempo de execução em função de diferentes tamanhos de entrada. Note que ele NÃO ESTÁ COMPLETO!

```cpp
Função ResolverVRPComDemanda(Locais, Demanda, C):
    // Locais: Lista de locais para entrega (excluindo o depósito)
    // Demanda: Dicionário mapeando cada local à sua demanda
    // C: Capacidade do veículo
    
    MelhorRota = NULL
    MenorCusto = INFINITO

    // Gera todas as combinações possíveis de rotas considerando a capacidade do veículo
    Combinações = GerarTodasAsCombinações(Locais)
    
    Para cada combinação em Combinações faça:
        Se VerificarCapacidade(combinação, Demanda, C) então:
            CustoAtual = CalcularCusto(combinação)
            Se CustoAtual < MenorCusto então:
                MenorCusto = CustoAtual
                MelhorRota = combinação
                
    Retornar MelhorRota, MenorCusto

Função GerarTodasAsCombinações(Locais):
    // Gera todas as permutações possíveis de locais e agrupa em rotas válidas conforme a capacidade
    // Esta função é bastante complexa, pois precisa considerar todas as subdivisões possíveis dos locais em rotas que atendam à capacidade do veículo
    // Retorna uma lista de combinações válidas
    Retornar combinações

Função VerificarCapacidade(Rota, Demanda, C):
    // Verifica se a demanda total da rota não excede a capacidade do veículo
    CargaTotal = 0
    Para cada local em Rota faça:
        CargaTotal += Demanda[local]
    Se CargaTotal > C então:
        Retornar Falso
    Senão:
        Retornar Verdadeiro

Função CalcularCusto(Rota):
    // Calcula o custo de uma rota com base na distância, tempo ou outro critério
    // O custo pode depender de fatores como a distância total percorrida, o número de veículos necessários, entre outros
    Retornar custo

// Início do programa
// Define os locais, suas demandas e a capacidade do veículo
Locais = [...]
Demanda = {...}
Capacidade = C

MelhorRota, MenorCusto = ResolverVRPComDemanda(Locais, Demanda, Capacidade)
Exibir "A melhor rota é:", MelhorRota, "com custo total de:", MenorCusto
```

%file 1.cpp

```cpp
#include <iostream>
#include <fstream>
#include <vector>
#include <string>
#include <sstream>
#include <algorithm>
#include <unordered_map>
#include <unordered_set>
#include <climits>

using namespace std;
vector<vector<int>> gerartodasCombinacoes(vector<vector<int>> locations, int max_stop, vector<int> places);
bool verificarCapacidade(vector<int> &route, vector<vector<int>> &demands, int capacity);





vector<int> resolverVRPComdemanda(vector<vector<int>> &locations, vector<vector<int>> &demands, int capacity, int max_stop) {
    //call the function gerartodasCombinacoes
    vector<int> bestRoute;
    int menor_custo = INT_MAX;
    vector<int> places;
    for (int i = 0; i < locations.size(); i++) {
        if (find(places.begin(), places.end(), locations[i][0]) != places.end()) {
            continue;
        }
        
        places.push_back(locations[i][0]);
        if (find(places.begin(), places.end(), locations[i][1]) != places.end()) {
            continue;
        }
        places.push_back(locations[i][1]);
    }
    sort(places.begin(), places.end());
    vector<vector<int>> allRoutes;
    vector<vector<int>> allRoutes_best_route;
    int j = 0;
    vector<vector<int>> current_routes;
    allRoutes = gerartodasCombinacoes(locations, max_stop, places);

    menor_custo = INT_MAX;



    for (int i = 0; i < allRoutes.size(); i++) {

        if (verificarCapacidade(allRoutes[i], demands, capacity)) {
            int custo = allRoutes[i][allRoutes[i].size() - 1];
            if (custo < menor_custo) {
                menor_custo = custo;
                bestRoute = allRoutes[i];
            }
        }


    }
    allRoutes_best_route.push_back(bestRoute);
    return bestRoute ;

}


typedef tuple<int, int, int> Edge;

int find_cost_back(int place, vector<vector<int>>&locations) {
    for (int i = 0; i < locations.size(); i++) {
        if (locations[i][0] == place && locations[i][1] == 0) {
            return locations[i][2];
        }
    }
    return 0;
}


// Function to perform DFS and find all paths from the origin to the destination
void dfs(int current, int destination, unordered_map<int, vector<pair<int, int>>>& graph,
         vector<int>& path, vector<vector<int>>& allPaths, unordered_set<int>& visited, vector<vector<int>> &locations,int weight = 0, int max_stop = 5, int ja_foi = 0, vector<int> places = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}) {
    // If the current node is the destination, and all nodes have been visited, then add the path to the list of all paths
    int visited_all = 1;
    //if all nodes have been visited and the current node is the destination, add the path to the list of all paths
    for (int i = 0; i < places.size(); i++) {
        if (visited.find(places[i]) == visited.end()) {
            visited_all = 0;
            break;
        }
    }
        

    if (current == destination && path.size() > 1 && visited_all == 1) {
        int weight_back_to_origin = find_cost_back(current, locations);
        //append the weight of the path
        path.push_back(weight+weight_back_to_origin);
        allPaths.push_back(path);
        path.pop_back();
    }
   
    if (path.size() > 1 && current != 0)
    {
    visited.insert(current);
    }

    // Recur for all the vertices adjacent to this vertex
    for (const auto& neighbor : graph[current]) {
        if (visited.find(neighbor.first) == visited.end()) {
            path.push_back(neighbor.first);
            dfs(neighbor.first, destination, graph, path, allPaths, visited, locations,weight + neighbor.second, max_stop, ja_foi,places);
            path.pop_back();
        }
    }

    // Remove the current node from visited set to allow other paths
    visited.erase(current);
}



// Function to find all paths from the origin to the destination
vector<vector<int>> findAllPaths(int origin, int destination, const vector<Edge>& edges, vector<vector<int>> &locations, int max_stop = 5) {
    // Build the graph from the edges
    unordered_map<int, vector<pair<int, int>>> graph;
    for (const auto& edge : edges) {
        int u = get<0>(edge);
        int v = get<1>(edge);
        int w = get<2>(edge);
        graph[u].emplace_back(v, w);
    }
    
    // Container to store all paths
    vector<vector<int>> allPaths;

    // Temporary path storage
    vector<int> path = {origin};

    // Set to keep track of visited nodes
    unordered_set<int> visited;
    vector<int> places;
    for (int i = 0; i < locations.size(); i++) {
        if (find(places.begin(), places.end(), locations[i][0]) != places.end()) {
            continue;
        }
        if (locations[i][0] == 0) {
            continue;
        }
        
        places.push_back(locations[i][0]);
        if (find(places.begin(), places.end(), locations[i][1]) != places.end()) {
            continue;
        }
        if (locations[i][1] == 0) {
            continue;
        }

        places.push_back(locations[i][1]);
    }
    sort(places.begin(), places.end());

    // Perform DFS to find all paths from origin to destination
    dfs(origin, destination, graph, path, allPaths, visited,locations, 0, max_stop,0, places  );
    vector<int> current_combination;

    return allPaths;
}


//function gerartodasCombinacoes receive the locations only and return all the possible routes
vector<vector<int>> gerartodasCombinacoes(vector<vector<int>> locations, int max_stop, vector<int> places) {
 vector<vector<int>> allRoutes;
    vector<int> currentRoute;
    int max_routes = 0;
    int origin = 0;
    int destination = 9 ;
    vector<Edge> edges;
    for (int i = 0; i < locations.size(); i++) {
        edges.push_back({locations[i][0], locations[i][1], locations[i][2]});
    }

    vector<int> all_origins;
    vector<int> all_destinations;
    for (int i = 0; i < locations.size(); i++) {
        //check if the origin is already in the all_origins
        if (find(all_origins.begin(), all_origins.end(), locations[i][0]) != all_origins.end()) {
            continue;
        }
        all_origins.push_back(locations[i][0]);
        
    }
    for (int i = 0; i < locations.size(); i++) {
        //check if the destination is already in the all_destinations
        if (find(all_destinations.begin(), all_destinations.end(), locations[i][1]) != all_destinations.end()) {
            continue;
        }
        all_destinations.push_back(locations[i][1]);
        
    }
    //find all paths from all origins to all destinations
        for (int j = 0; j < all_destinations.size(); j++) {
            vector<vector<int>> allPaths = findAllPaths(0, all_destinations[j], edges, locations, max_stop);
            //check if the route is valid(have all places), if not remove it
            
            for (int k = 0; k < allPaths.size(); k++) {
                allRoutes.push_back(allPaths[k]);
            }
        }


    return allRoutes;
}
int acha_valor(vector<vector<int>> &demands, int id) {
    for (int i = 0; i < demands.size(); i++) {
        if (demands[i][0] == id) {
            return demands[i][1];
        }
    }
    return 0;
}

//function verificarCapacidade receive the route, demands and capacity of the vehicle
bool verificarCapacidade(vector<int> &route, vector<vector<int>> &demands, int capacity) {
    int carga_atual = 0;
    for (int i = 0; i < route.size(); i++) {
        carga_atual = carga_atual + acha_valor(demands, route[i]);
        if (route[i] == 0) {
            carga_atual = 0;
        }
        if (carga_atual > capacity) {
        return false;
    }

    }
    
    return true;
}


int main() {
    //   locations
    vector<vector<int>> locations;
    //vetor demands
    vector<vector<int>> demands;


    //capacity of the vehicle
    int capacity;
    // best route
    vector<int> bestRoute;
    // best cost
    int bestCost = 0;
    //read the file
    ifstream file("grafo.txt");
    string line;
    int n;
    getline(file, line);
    stringstream ss(line);
    ss >> n;
    // cout << n << endl;
    //read the demands
    for (int i = 0; i < n -1; i++) {
        getline(file, line);
        stringstream ss(line);
        vector<int> demand;
        int demanda;
        int id;
        ss >> id;
        ss >> demanda;
        demand.push_back(id);
        demand.push_back(demanda);
        demands.push_back(demand);

    }
    int arrestas;
    getline(file, line);
    stringstream ss1(line);
    ss1 >> arrestas;
    // cout << arrestas << endl;
    //read the locations
    for (int i = 0; i < arrestas; i++) {
        getline(file, line);
        stringstream ss(line);
        vector<int> location;
        int x, y, weight;
        ss >> x >> y >> weight;
        // cout << x << " " << y << " " << weight << endl;
        location.push_back(x);
        location.push_back(y);
        location.push_back(weight);
        locations.push_back(location);
    }




    int max_stop = 5;
    capacity = 15;
    //call the function resolverVRPComdemanda
    vector<int> allRoutes_best_route;
    vector<int> already_visited;
   allRoutes_best_route = resolverVRPComdemanda(locations, demands, capacity, max_stop);

    //split the best route when the value is 0
    vector<vector<int>> allRoutes;
    vector<int> currentRoute;
    for (int i = 0; i < allRoutes_best_route.size()-1; i++) {
        if (allRoutes_best_route[i] == 0) {
            currentRoute.push_back(allRoutes_best_route[i]);
            allRoutes.push_back(currentRoute);
            currentRoute.clear();
            currentRoute.push_back(allRoutes_best_route[i]);
        } else {
            currentRoute.push_back(allRoutes_best_route[i]);
        }
    }
    //remove the last element of the vector
    

    int size;
    //print the best route
    for (int i = 0; i < allRoutes.size(); i++) {
        if (allRoutes[i].size() < 2) {
            continue;
        }   
        size = allRoutes[i].size() - 1;
        cout << "[";
        for (int j = 0; j < (size); j++) {
            cout << allRoutes[i][j] << " ";
        }
        cout << allRoutes[i][size] << "],";
        cout << endl;
    }
    cout << "Weight: " << allRoutes_best_route[allRoutes_best_route.size() - 1] << endl;
}
 

```


### 2. Solução aproximada

Para problemas complexos as heurísticas são ferramentas essenciais, pois oferecem soluções aproximadas em tempo razoável, especialmente quando o problema é grande demais para abordagens de força bruta ou métodos exatos. Heurísticas não garantem a melhor solução possível, mas frequentemente encontram soluções muito boas com um esforço computacional significativamente menor.

Algumas das heurísticas e métodos meta-heurísticos mais comuns para o VRP incluem:

  - _Heurística de Inserção Mais Próxima_: Começa com uma rota contendo apenas o depósito e, em seguida, iterativamente insere o cliente mais próximo em sua posição de menor custo até que todos os clientes sejam incluídos. Esta abordagem é simples e relativamente rápida, mas pode não encontrar a melhor solução possível.
  - _Heurística de Clarke e Wright (Economias)_: Uma das heurísticas mais conhecidas para o VRP, que começa com cada cliente em sua própria rota e, em seguida, progressivamente mescla rotas que resultam nas maiores "economias", ou seja, reduções no custo total da rota. Essa abordagem tenta equilibrar o custo de visitar cada cliente com o custo de conectar rotas.
  - _Algoritmos Genéticos_: Uma classe de métodos de busca e otimização baseados nos princípios de seleção natural e genética. Eles usam operações como seleção, cruzamento e mutação para evoluir uma população de soluções ao longo do tempo, em busca da melhor solução possível.

Implemente uma das heurísticas citadas, ou elabore outra solução aproximada, que dê uma solução em tempo razoável para o problema. Aqui, além de implementar o algoritmo, é importante que você descreva brevemente sua estratégia de solução no relatório.

Mesmo cortes mais simples do espaço de busca estão valendo, por exemplo, considerando que há um limite máximo de visitas em cada rota, podemos de cara cortar as combinações de cidades que ultrapassam esse limite. Podemos, ainda, estabelecer um limite mínimo, para evitar rotas pequenas.





### 3. Soluções paralelizadas

Na abordagem de força bruta para a resolução do problema de otimização de rotas de veículos, a parte mais custosa computacionalmente é a geração e avaliação de todas as permutações possíveis das cidades (ou pontos de entrega).

Esta é também a parte que oferece maior potencial para paralelização. Mas existem outras?

Nesta etapa do projeto você deve realizar 2 implementações paralelas da sua solução, sendo:

- 1 solução baseada em "paralelização local", ou seja, usar threads para extrair o máximo de proveito da máquina que rodará o programa;
- 1 solução baseada em "paralelização global", ou seja, construir uma solução que paraleliza a execução em múltiplas máquinas de um cluster, acelerando ainda mais a execução. Note que seu programa de paralelização global deve também fazer uso de paralelização local. Em outras palavras, você particionará o problema e alocará cada parte em uma máquina; em cada máquina você usará os recursos presentes (cores) para paralelizar ainda mais!


**_NOTA_**: Compare todas as suas soluções em termos de tempo de execução. Para a mesma entrada, execute-a com cada uma das 4 implementações e compare os tempos de execução.

## Critérios de avaliação

A correção do projeto levará em conta:

+ [Até 1 pontos] **Organização geral**
  - Organização do código fonte e/ou do notebook que centraliza seus códigos. Se a entrega for via jupyter notebook, ele deve conter células markdown que guiem a apreciação do trabalho. Caso seu código esteja organizado em arquivos ".cpp", ".py", etc, submeta também um relatório em PDF descrevendo seu trabalho;

+ [Até 4 pontos] **Implementações**
  - Sua entrega deve conter ao menos 3 implementações (uma para cada solução: exaustiva, OpenMP e MPI). Soluções extras nestas vertentes são encorajadas e serão valorizadas;
  - A parte de MPI deve ser obrigatoriamente executada no cluster. Portanto, submeta também os arquivos de configuração dos executáveis em batch;
  - Esperamos códigos seguindo as boas práticas de implementação e devidamente comentados;

+ [Até 3 pontos] **Avaliação de resultados**
  - Comparações das abordagens com tamanhos diferentes de grafos;
  - Clareza na comunicação dos resultados. Ex: gráficos e/ou pequenos textos que descrevem os resultados obtidos;
  - Justificativas: é esperado que a execução paralela seja mais rápida que a sequencial, assim como uma heurística seja mais rápida que a abordagem exaustiva. Caso seus resultados diferem do esperado, argumente potenciais causas;

+ [Até 1 ponto] **Extras**
  - A rubrica não cita o máximo de implementações a ser feita. Caso você opte por fazer algo além das 4 solicitadas, daremos até 1 ponto extra na nota do trabalho pelo esforço;


